# Building ML-models with feature selection by filter combinations (strict CoV filter versions)

Preprocessing and feature selection is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

In [1]:
#import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from functools import partial #to pass parameters to function inside another function
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
#import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in mutual information)
seed = 0

## Load and clean data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features
X = gexp

#define targets and extract their fluxes
target_labels = ['R_PPC', 'R_AKGDH', 'R_ICDHyr']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose selection parameters:

In [5]:
cov_filters = [0.5, 0.9]
k_numbers = [10,20]

Choose learning algorithms:

In [6]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'lasso_a0.05': linear_model.Lasso(alpha = 0.05),
    'ridge_a0.05': linear_model.Ridge(alpha = 0.05)
    })

Make combinations:

In [7]:
#make combinations of learning algorithms and filters 
def get_pipelines(algorithm, cov_filters, k_numbers):
    models = dict()
    
    for cov_filter in cov_filters:
        
        # filter by CoV only
        steps = [
            ('filter', CoVSelector(p = cov_filter)),
            ('standardize', StandardScaler()),
            ('m', algorithm)
        ]
        models['v%.0f' % (cov_filter*100)] = Pipeline(steps=steps)        

        # filter by CoV and SelectKBest
        for k in k_numbers:
            
            #selection by mi
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = partial(
                    mutual_info_regression, 
                    random_state=seed), 
                                          k = k)), 
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_mi%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)

            #selection by f
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = f_regression, 
                                          k = k)),
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_f%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)
            
    return models

In [8]:
models = dict()

for name, algorithm in algorithms.items():
     models[name]= get_pipelines(algorithm, cov_filters = cov_filters, k_numbers = k_numbers)

## Learning

In [9]:
# evaluate the models and store results in nested dicts: target -> learning algorithm -> scores/preds of all selection combinations
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        model_scores[model_name] = compare_models(pipelines, X, y)
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.209e-03, tolerance: 5.633e-04
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.209e-03, tolerance: 5.633e-04
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [10]:
scores_filter_strict = scores
preds_filter_strict = preds
%store scores_filter_strict
%store preds_filter_strict

Stored 'scores_filter_strict' (dict)
Stored 'preds_filter_strict' (dict)


### Scores and predictions

In [11]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm, '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each combination of selection methods:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'Selection combo'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each combination of selection methods:')
        df = scores[y][algorithm][0]
        df.columns.name = 'Selection combo'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each combination of selection methods (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'Selection combo'
        display(df)


R_PPC	(target)
lr 	(learning algorithm)
–> summary of all cv split scores for each combination of selection methods:


Score,average,std
Selection combo,,
v50,0.277279,0.310830
v50_f20,0.410882,0.664849
v50_mi20,0.492968,0.826446
v90,0.547302,0.840779
v50_f10,0.598956,0.973113
v90_mi10,0.988003,1.533409
v90_f10,0.990921,1.598316
v90_mi20,1.003002,1.598467
v90_f20,1.076619,1.973893



R_PPC	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v50,v50_mi10,v50_f10,v50_mi20,v50_f20,v90,v90_mi10,v90_f10,v90_mi20,v90_f20
Test set,,,,,,,,,,
Acetate,0.191031,2.745166,0.608406,0.182148,0.477095,0.132563,0.328653,0.121640,0.536065,0.748998
Fructose,0.299097,0.197310,0.301007,0.377689,0.398427,0.334158,0.134304,0.038025,0.281687,0.039867
Galactose,1.062181,5.336438,3.136492,2.654502,2.115252,2.733386,5.019886,5.115509,5.205812,6.266600
Glucose,0.122427,0.108536,0.112142,0.042349,0.028318,0.103383,0.582737,0.129579,0.402267,0.250710
Glycerol,0.060724,0.603862,0.106875,0.378383,0.068784,0.584657,0.276100,1.184401,0.403956,0.375438
Gluconate,0.304691,0.008043,0.283155,0.150957,0.001585,0.272837,0.497107,0.447443,0.421643,0.545294
Pyruvate,0.145804,0.051801,0.203630,0.024804,0.075564,0.095289,0.353255,0.231967,0.054280,0.061919
Succinate,0.032279,0.066188,0.039944,0.132914,0.122032,0.122144,0.711983,0.658802,0.718307,0.324125



R_PPC	(target)
lr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


...

Score,average,std
Selection combo,,
v50_f20,0.342765,0.380505
v50_f10,0.574820,0.615170
v50,0.598491,0.967256
v50_mi20,0.606897,0.747463
v50_mi10,0.721395,0.970150
v90_mi20,1.216324,1.667284
v90_f20,1.321136,2.073741
v90_mi10,1.371690,2.011273
v90,1.550364,2.704025



R_ICDHyr	(target)
ridge_a0.05	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v50,v50_mi10,v50_f10,v50_mi20,v50_f20,v90,v90_mi10,v90_f10,v90_mi20,v90_f20
Test set,,,,,,,,,,
Acetate,0.046604,0.097471,0.121313,0.065226,0.134407,0.220040,0.133567,0.236014,0.191478,0.063714
Fructose,0.173551,0.343034,0.159781,0.158046,0.236238,0.021411,0.309193,0.129298,0.163226,0.177679
Galactose,3.137373,3.185062,1.392346,2.227515,0.108853,8.398552,6.337878,10.393176,5.300279,6.571741
Glucose,0.272488,0.425904,0.523707,0.332642,0.365453,0.083938,1.067457,0.750782,0.935980,0.894705
Glycerol,0.150058,0.260317,0.124030,0.192959,0.039843,0.320162,0.038552,0.007655,0.260155,0.102851
Gluconate,0.247785,0.995203,1.796829,1.451545,1.297891,2.550985,2.385915,5.781049,2.163524,2.003854
Pyruvate,0.497917,0.424048,0.410905,0.412379,0.414451,0.716239,0.512716,0.486616,0.494167,0.503444
Succinate,0.262154,0.040120,0.069647,0.014867,0.144983,0.091581,0.188237,0.119655,0.221782,0.251105



R_ICDHyr	(target)
ridge_a0.05	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


Selection combo,actual,v50,v50_mi10,v50_f10,v50_mi20,v50_f20,v90,v90_mi10,v90_f10,v90_mi20,v90_f20
Test set,,,,,,,,,,,
Acetate,4.696417,4.477543,4.238650,5.266156,4.390087,4.065187,3.663017,4.069129,5.804837,3.797157,4.397192
Fructose,4.566119,5.358571,6.132454,5.295696,5.287775,5.644811,4.663883,5.977933,5.156508,5.311428,5.377422
Galactose,0.496177,2.052871,2.076533,1.187028,1.601420,0.550188,4.663349,3.640890,5.653037,3.126056,3.756927
Glucose,2.977971,2.166508,1.709643,1.418388,1.987372,1.889663,3.227937,-0.200884,0.742163,0.190648,0.313565
Glycerol,2.464836,2.094969,3.106475,2.159123,1.989225,2.366630,3.253984,2.369812,2.445967,3.106076,2.718347
Gluconate,1.154034,1.439986,2.302531,3.227634,2.829166,2.651843,4.097956,3.907460,7.825558,3.650814,3.466548
Pyruvate,7.979320,4.006284,4.595702,4.700575,4.688820,4.672286,2.264219,3.888192,4.096458,4.036206,3.962181
Succinate,3.038949,2.242277,2.917027,3.250603,3.084129,3.479544,3.317260,2.466906,2.675322,2.364964,2.275855
